In [ ]:
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_lg

In [1]:
import pandas as pd
from datetime import datetime
import spacy_transformers
import json
import numpy as np
import spacy

# Storing docs in binary format
from spacy.tokens import DocBin

In [2]:
# Loading the Slang terms
with open('bigSlangList.json') as f:
    slang = json.load(f)

In [16]:
# an example of creating a pattern for NER
nlp = spacy.load('en_core_web_lg')
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "GENZ_TERM", "pattern": slang} for slang in slang.keys()]
with nlp.select_pipes(enable="tagger"):
    ruler.add_patterns(patterns)

In [ ]:

with open('teenDict.json') as f:
    data = json.load(f)

overallTeenList = []
for key in data:
    overallTeenList.append(data[key][0])
    overallTeenList.append(data[key][1])
    for x in data[key][2]:
        overallTeenList.append(x)

print(len(overallTeenList))
with open('adultDict.json') as f:
    data = json.load(f)

overallAdultList = []
for key in data:
    overallAdultList.append(data[key][0])
    overallAdultList.append(data[key][1])
    for x in data[key][2]:
        overallAdultList.append(x)

print(len(overallAdultList))
teenDf = pd.DataFrame(overallTeenList)
adultDf = pd.DataFrame(overallAdultList)

In [ ]:
teenDf['label'] = 0
adultDf['label'] = 1

In [ ]:
mergedDf = pd.concat([teenDf, adultDf])
mergedDf.columns = ['text', 'label']

In [ ]:
mergedDf.reset_index(drop=True, inplace=True)

In [ ]:
mergedDf.head()

In [ ]:
from tokenizer import CrazyTokenizer
tokenizer = CrazyTokenizer(remove_punct=True, remove_breaks=True, ignore_stopwords=False, ignore_quotes=False, decontract= True, reddit_usernames='', urls='', subreddits='', latin_chars_fix=True, hashtags='split', pos_emojis=True, neg_emojis=True, neutral_emojis=True,  drop_nums='')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_idx, test_idx in sss.split(mergedDf, mergedDf['label']):
    reddit_train_set = mergedDf.loc[mergedDf.index.intersection(
        train_idx)]
    reddit_test_set = mergedDf.loc[mergedDf.index.intersection(
        test_idx)]

In [ ]:
# SKIP IF YOU WANT TO USE THE PRE-PROCESSED DATA
reddit_train_set['text'] = reddit_train_set['text'].apply(tokenizer.tokenize)
reddit_test_set['text'] = reddit_test_set['text'].apply(tokenizer.tokenize)

In [ ]:
# SKIP IF YOU WANT TO USE THE PRE-PROCESSED DATA
reddit_train_set['text'] = reddit_train_set['text'].apply(lambda x: ' '.join(x))
reddit_test_set['text'] = reddit_test_set['text'].apply(lambda x: ' '.join(x))

In [ ]:
# SKIP IF YOU WANT TO USE THE PRE-PROCESSED DATA
reddit_train_set.to_csv('reddit_train_set.csv', index=False)
reddit_test_set.to_csv('reddit_test_set.csv', index=False)

In [ ]:
# SKIP IF YOU WANT TO USE THE PRE-PROCESSED DATA
reddit_train_set.head()

In [3]:
# START HERE TO USE THE PRE-PROCESSED DATA
reddit_test_set = pd.read_csv('reddit_test_set.csv')
reddit_train_set = pd.read_csv('reddit_train_set.csv')

In [4]:
reddit_train_set.reset_index(drop=True, inplace=True)
reddit_test_set.reset_index(drop=True, inplace=True)

In [5]:
reddit_train_set.head()

,text,label
0,my year old grandmother replied ok boomer to m...,0
1,we forgot about our greatest ally the silent g...,0
2,to anyone who does not know this person is gra...,0
3,proof that the silent generation is not really...,0
4,best grandma of,0


In [7]:
print(len(reddit_train_set))
for i in range(len(reddit_train_set['text'])):
    if type(reddit_train_set['text'][i]) == float:
        reddit_train_set.drop(index=i, inplace=True)
print(len(reddit_train_set))

33064
32525


In [8]:
print(len(reddit_test_set))
for i in range(len(reddit_test_set['text'])):
    if type(reddit_test_set['text'][i]) == float:
        reddit_test_set.drop(index=i, inplace=True)
print(len(reddit_test_set))

14171
13962


In [9]:
# Checking that there's a pretty even split given the amount of data for both labels
reddit_test_set[5000:]['label'].value_counts()

0    5077
1    3885
Name: label, dtype: int64

In [11]:
trainDataset = reddit_train_set['text'].tolist()
testDataset = reddit_test_set['text'].tolist()

In [12]:
traindataset = list(reddit_train_set[["text", "label"]].sample(frac=1).itertuples(index=False, name=None))
testdataset = list(reddit_test_set[["text", "label"]].sample(frac=1).itertuples(index=False, name=None))
devdataset = testdataset[:5000]
testdataset2= testdataset[5000:]

In [13]:
print(len(testdataset2))

8962


In [17]:

from spacy.matcher import Matcher, PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

patterns = [nlp.make_doc(slang) for slang in slang.keys()]

matcher.add("genz", patterns)

In [ ]:
doc = nlp('hello ur the coolest boomer')

In [ ]:
matcher(doc)

In [18]:
import tqdm
from spacy.language import Language
from spacy.tokens import Doc
from spacy.matcher import PhraseMatcher
from spacy.language import Language
import spacy
import spacy
from spacy.training import Example
from spacy.pipeline import EntityRecognizer
from spacy.tokens import Span, DocBin
# Much of the data set forming code is from https://towardsdatascience.com/improving-the-ner-model-with-patent-texts-spacy-prodigy-and-a-bit-of-magic-44c86282ea99

In [ ]:

def convert(data, outfile):
    LABEL = "GENZ_TERM"
    db = DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        ents = []

        for match_id, start, end in matcher(doc):
            span = Span(doc, start, end, label=LABEL)
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)

        filtered_ents =  spacy.util.filter_spans(ents)
        doc.ents = filtered_ents
        doc.cats["ADULT"] = label == 1
        doc.cats["GENZ"] = label == 0
        db.add(doc)
    
    db.to_disk(outfile)
convert(traindataset, "train.spacy")
convert(devdataset, "dev.spacy")
convert(testdataset2, "test.spacy")

In [22]:
def convertbaseline(data, outfile):
    LABEL = "GENZ_TERM"
    db = DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        ents = []

        for match_id, start, end in matcher(doc):
            span = Span(doc, start, end, label=LABEL)
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)

        filtered_ents =  spacy.util.filter_spans(ents)
        doc.ents = filtered_ents
        doc.cats["GENZ"] = label == 1
        doc.cats["GENZ"] = label == 0
        db.add(doc)
    
    db.to_disk(outfile)
    
convertbaseline(testdataset2, "baseline.spacy")

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model

In [120]:
!python -m spacy evaluate ./model/model-best/ ./test1.spacy --output robertaModel 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Using CPU
Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors

================================== Results ==================================

TOK                 100.00
NER P               97.35 
NER R               87.58 
NER F               92.21 
TEXTCAT (macro F)   80.42 
SPEED               875   


=============================== NER (per type) ===============================

                P       R       F
GENZ_TERM   97.35   87.58   92.21


=========================== Textcat F (per label) ===========================

         

In [23]:
!python -m spacy evaluate ./model/model-best/  ./baseline.spacy

ℹ Using CPU
Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors

================================== Results ==================================

TOK                 100.00
NER P               97.08 
NER R               86.38 
NER F               91.42 
TEXTCAT (macro F)   44.53 
SPEED               788   


=============================== NER (per type) ===============================

                P       R       F
GENZ_TERM   97.08   86.38   91.42


=========================== Textcat F (per label) ===========================

             P       R       F
ADULT     0.00    0.00    0.00
GENZ    100.00   80.28   89.06


======================== Textcat ROC AUC (per label) ========================

        ROC AUC
ADULT      None
GENZ       0.90



In [96]:
!python -m spacy train configTry2.cfg --output ./spacy_output4 --paths.train ./train1.spacy --paths.dev ./dev1.spacy 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Saving to output directory: spacy_output4
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-21 22:58:52,339] [INFO] Set up nlp object from config
[2023-04-21 22:58:52,348] [INFO] Pipeline: ['transformer', 'ner', 'textcat']
[2023-04-21 22:58:52,351] [INFO] Created vocabulary
^C


In [121]:
!python -m spacy evaluate ./spacy_output4/model-best/ ./test1.spacy --output XLNNetModel

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Using CPU

================================== Results ==================================

TOK                 100.00
NER P               90.22 
NER R               80.69 
NER F               85.19 
TEXTCAT (macro F)   79.18 
SPEED               615   


=============================== NER (per type) ===============================

                P       R       F
GENZ_TERM   90.22   80.69   85.19


=========================== Textcat F (per label) ===========================

            P       R       F
ADULT   69.30   70.54   69.91
GENZ    88.75   88.15   88.45


======================== Textcat ROC AUC (per label) ========================

        ROC AUC
ADUL

In [24]:
!python -m spacy evaluate ./spacy_output4/model-best/ ./baseline.spacy

ℹ Using CPU

================================== Results ==================================

TOK                 100.00
NER P               90.62 
NER R               79.57 
NER F               84.73 
TEXTCAT (macro F)   41.87 
SPEED               484   


=============================== NER (per type) ===============================

                P       R       F
GENZ_TERM   90.62   79.57   84.73


=========================== Textcat F (per label) ===========================

             P       R       F
ADULT     0.00    0.00    0.00
GENZ    100.00   72.04   83.75


======================== Textcat ROC AUC (per label) ========================

        ROC AUC
ADULT      None
GENZ       0.88



In [2]:
# Much of the framework for the displaying code is found here https://towardsdatascience.com/improving-the-ner-model-with-patent-texts-spacy-prodigy-and-a-bit-of-magic-44c86282ea99
import spacy
# test out the main model (roberta)
nlp = spacy.load("./model/model-best")
# normal english - adult label
doc = nlp("Fish, a diverse and fascinating group of aquatic animals, play a critical role in the health of our planet's ecosystems. From the colorful and ornamental specimens found in home aquariums to the vast schools that populate the oceans, fish are both aesthetically pleasing and biologically important. They serve as a vital food source for humans and other animals, and many species are also used in medical research.")


In [3]:
print(doc.cats,  "-",  doc.text)

{'ADULT': 0.892070472240448, 'GENZ': 0.10792949795722961} - Fish, a diverse and fascinating group of aquatic animals, play a critical role in the health of our planet's ecosystems. From the colorful and ornamental specimens found in home aquariums to the vast schools that populate the oceans, fish are both aesthetically pleasing and biologically important. They serve as a vital food source for humans and other animals, and many species are also used in medical research.


In [4]:
from spacy import displacy

In [5]:
# test out the main model (roberta)
# Gen Z slang
doc = nlp("Silly goofy moods for the greatest silly goofy gals out there! bffr! i really am about to yeet myself after that took forever to run on the computer and i think that ur a really good friend live love laugh!!!")

In [6]:
# test out the main model (roberta)
# Gen Z slang
doc1 = nlp("Kelly Clarkson's show was literally straight fire")
# debatable Gen Z slang but it's also a normal English sentence
doc2 = nlp("Kelly Clarkson's show last night was pretty good")
# Was one of the Adult sentences from the dataset
doc3 = nlp("Watching the election results roll in is like showing up at a party you were super stoked for, only to find that there's not only no alcohol but it's not even a party, but a colonoscopy without anesthesia")

In [7]:
# test out the main model (roberta)
colors = {"GENZ_TERM": "#F67DE3"}
options = {"colors": colors}
# import displacy
spacy.displacy.render(doc, style="ent", options=options, jupyter=True)
print(doc.cats,  "-",  doc.text)
spacy.displacy.render(doc1, style="ent", options=options, jupyter=True)
print(doc1.cats,  "-",  doc1.text)
spacy.displacy.render(doc2, style="ent", options=options, jupyter=True)
print(doc2.cats,  "-",  doc2.text)
spacy.displacy.render(doc3, style="ent", options=options, jupyter=True)
print(doc3.cats,  "-",  doc3.text)

{'ADULT': 0.08973050862550735, 'GENZ': 0.9102694392204285} - Silly goofy moods for the greatest silly goofy gals out there! bffr! i really am about to yeet myself after that took forever to run on the computer and i think that ur a really good friend live love laugh!!!


{'ADULT': 0.062104884535074234, 'GENZ': 0.9378951191902161} - Kelly Clarkson's show was literally straight fire


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


{'ADULT': 0.13191844522953033, 'GENZ': 0.8680815696716309} - Kelly Clarkson's show last night was pretty good


{'ADULT': 0.7686761617660522, 'GENZ': 0.23132386803627014} - Watching the election results roll in is like showing up at a party you were super stoked for, only to find that there's not only no alcohol but it's not even a party, but a colonoscopy without anesthesia


In [8]:
# test out the main model (roberta)
# gen z slang
doc4 = nlp("have you seen those sick new photos of the glow-up jellyfish? They're lit AF!")
# Adult sentence
doc5 = nlp("Have you had the opportunity to view recent captivating images of bioluminescent jellyfish, which are visually striking and awe-inspiring?")

In [9]:
# test out the main model (roberta)
spacy.displacy.render(doc4, style="ent", options=options, jupyter=True)
print(doc4.cats,  "-",  doc4.text)
spacy.displacy.render(doc5, style="ent", options=options, jupyter=True)
print(doc5.cats,  "-",  doc5.text)

{'ADULT': 0.08787418156862259, 'GENZ': 0.912125825881958} - have you seen those sick new photos of the glow-up jellyfish? They're lit AF!


{'ADULT': 0.5239931344985962, 'GENZ': 0.4760068655014038} - Have you had the opportunity to view recent captivating images of bioluminescent jellyfish, which are visually striking and awe-inspiring?


In [10]:
# test out the main model (roberta)
# Gen Z slang
doc = nlp("lol why did u text me like that im lowkey smashed rn")
print(doc.cats,  "-",  doc.text)
spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

{'ADULT': 0.022113226354122162, 'GENZ': 0.9778867363929749} - lol why did u text me like that im lowkey smashed rn


In [11]:
import spacy
# Test out the XLNet model
nlp2 = spacy.load("./spacy_output4/model-best")
# gen z slang
doc = nlp2("Silly goofy moods for the greatest silly goofy gals out there! bffr! i really am about to yeet myself after that took forever to run on the computer and i think that ur a really good friend live love laugh!!!")

In [24]:
# Test out the XLNet model
# gen z slang
doc1 = nlp2("Kelly Clarkson's show was literally straight fire")
# debatable Gen Z slang but it's also a normal English sentence
doc2 = nlp2("Kelly Clarkson's show last night was pretty good")
# Was one of the Adult sentences from the dataset
doc3 = nlp2("Watching the election results roll in is like showing up at a party you were super stoked for, only to find that there's not only no alcohol but it's not even a party, but a colonoscopy without anesthesia")

In [25]:
# Test out the XLNet model
# gen z slang
doc4 = nlp2("Dude, have you seen those sick new photos of the glow-up jellyfish? They're lit AF!")
# Adult sentence
doc5 = nlp2("Have you had the opportunity to view recent captivating images of bioluminescent jellyfish, which are visually striking and awe-inspiring?")

In [26]:
# Test out the XLNet model
spacy.displacy.render(doc1, style="ent", options=options, jupyter=True)
print(doc1.cats,  "-",  doc1.text)
spacy.displacy.render(doc2, style="ent", options=options, jupyter=True)
print(doc2.cats,  "-",  doc2.text)
spacy.displacy.render(doc3, style="ent", options=options, jupyter=True)
print(doc3.cats,  "-",  doc3.text)
spacy.displacy.render(doc4, style="ent", options=options, jupyter=True)
print(doc4.cats,  "-",  doc4.text)
spacy.displacy.render(doc5, style="ent", options=options, jupyter=True)
print(doc5.cats,  "-",  doc5.text)

{'ADULT': 0.16280242800712585, 'GENZ': 0.8371975421905518} - Kelly Clarkson's show was literally straight fire


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


{'ADULT': 0.2908596992492676, 'GENZ': 0.7091403007507324} - Kelly Clarkson's show last night was pretty good


{'ADULT': 0.8968883752822876, 'GENZ': 0.10311158001422882} - Watching the election results roll in is like showing up at a party you were super stoked for, only to find that there's not only no alcohol but it's not even a party, but a colonoscopy without anesthesia


{'ADULT': 0.13048189878463745, 'GENZ': 0.8695181012153625} - Dude, have you seen those sick new photos of the glow-up jellyfish? They're lit AF!


{'ADULT': 0.7918338775634766, 'GENZ': 0.20816615223884583} - Have you had the opportunity to view recent captivating images of bioluminescent jellyfish, which are visually striking and awe-inspiring?


In [15]:
# gen z slang
doc = nlp2("lol why did u text me like that im lowkey smashed rn")
print(doc.cats,  "-",  doc.text)
spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

{'ADULT': 0.09943749755620956, 'GENZ': 0.9005624651908875} - lol why did u text me like that im lowkey smashed rn


In [16]:
# adult label
doc = nlp2("Fish, a diverse and fascinating group of aquatic animals, play a critical role in the health of our planet's ecosystems. From the colorful and ornamental specimens found in home aquariums to the vast schools that populate the oceans, fish are both aesthetically pleasing and biologically important. They serve as a vital food source for humans and other animals, and many species are also used in medical research.")
print(doc.cats,  "-",  doc.text)
spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

{'ADULT': 0.8963122367858887, 'GENZ': 0.10368771851062775} - Fish, a diverse and fascinating group of aquatic animals, play a critical role in the health of our planet's ecosystems. From the colorful and ornamental specimens found in home aquariums to the vast schools that populate the oceans, fish are both aesthetically pleasing and biologically important. They serve as a vital food source for humans and other animals, and many species are also used in medical research.


In [19]:
# code block taken from https://stackoverflow.com/questions/72414166/how-is-it-possible-to-use-the-spacytransformers-model-in-the-transfomers-pipel
# Mentioned in the paper as well
import spacy
import os
nlp = spacy.load("model/model-best")
output_dir = 'hf-model-output-dir3'
os.makedirs(output_dir, exist_ok=True)


from transformers import PreTrainedTokenizerFast, RobertaTokenizer, RobertaForMaskedLM, AutoTokenizer

# Convert spaCy tokenization to your model's standard tokenization (eg. wordpiece, bpe, etc.)

class CustomTokenizer(PreTrainedTokenizerFast):
    def __init__(self, spacy_tokenizer, backend_tokenizer, *args, **kwargs):
        super().__init__(tokenizer_object=backend_tokenizer, *args, **kwargs)
        self.spacy_tokenizer = spacy_tokenizer
        self._backend_tokenizer = backend_tokenizer

    def _tokenize(self, text):
        return [token.text for token in self.spacy_tokenizer(text)]

    def __getattr__(self, name):
        return getattr(self._backend_tokenizer, name)

    @property
    def backend_tokenizer(self):
        return self._backend_tokenizer

    def save_pretrained(self, save_directory, legacy_format=True, filename_prefix=None, push_to_hub=False, **kwargs):
        self._backend_tokenizer.save_pretrained(save_directory, legacy_format=legacy_format, filename_prefix=filename_prefix, push_to_hub=push_to_hub, **kwargs)


# Instantiate the custom tokenizer with the spaCy tokenizer and a backend tokenizer

spacy_tokenizer = nlp.tokenizer
backend_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
custom_tokenizer = CustomTokenizer(spacy_tokenizer, backend_tokenizer)

# Save the tokenizer

custom_tokenizer.save_pretrained(output_dir)

# Save the model weights and configuration files
nlp.config.to_disk(os.path.join(output_dir, 'config.json'))
import spacy
from transformers import AutoConfig
import json


# Get the label names from the named entity recognizer component
ner = nlp.get_pipe("textcat")
label_names = ner.labels


# Create an AutoConfig object based on the spaCy model 
#config = AutoConfig.from_pretrained('roberta-base', num_labels=len(label_names), id2label={i: label for i, label in enumerate(label_names)}, label2id={label: i for i, label in enumerate(label_names)})
config = AutoConfig.from_pretrained('roberta-base')

# Save the configuration to disk in the Transformers-compatible format
config_dict = config.to_dict()
with open(os.path.join(output_dir, 'config.json'), 'w') as f:
    json.dump(config_dict, f)

nlp.vocab.to_disk(os.path.join(output_dir, 'vocab.txt'))
from transformers import RobertaForTokenClassification

# Create a Hugging Face model using the configuration object

hf_model = RobertaForMaskedLM.from_pretrained("roberta-base", config=config)

# The spaCy model doesn't have a position embedding tensor that the Hugging Face model expects. And the 
# Hugging Face model has a pooler layer that the spaCy model does not have. To fix this, I had to exclude the pooler 
# layer and craftily add a position embedding tensor into the hf output. As a result, c/s scores will be lower. And not
# to mention the headache of converting the tokenizer.

# Get the weights from the spaCy model and set the Hugging Face model weights
state_dict = {k.replace("roberta.", ""): v for k, v in nlp.get_pipe("transformer").model.transformer.named_parameters()}
state_dict["embeddings.position_ids"] = hf_model.roberta.embeddings.position_ids
state_dict = {k: v for k, v in state_dict.items() if not k.startswith("pooler.")}
#state_dict = {k: v for k, v in state_dict.items()}
hf_model.roberta.load_state_dict(state_dict)


# Finally, save the Hugging Face model to disk

hf_model.save_pretrained(output_dir)

In [21]:
from transformers import PreTrainedTokenizerFast, RobertaTokenizer, RobertaForMaskedLM, AutoTokenizer, AutoConfig
config = AutoConfig.from_pretrained('roberta-base')
tokeyboi = AutoTokenizer.from_pretrained("./hf-model-output-dir3", config=config)
modelboi = RobertaForMaskedLM.from_pretrained("./hf-model-output-dir3", config=config)


In [23]:
# this is the attempt at some Masked Language Model learning for the converted huggingface model with the roberta model
import torch

text = "A Gen Z person might say lol ur <mask> is so janky. "
inputs = tokeyboi(text, return_tensors="pt")
token_logits = modelboi(**inputs).logits

predicted_token_class_ids = token_logits.argmax(-1)

mask_token_index = torch.where(inputs["input_ids"] == predicted_token_class_ids)[1]

mask_token_logits = token_logits[0, mask_token_index, :]

top5Tokes = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top5Tokes:
    print(f"'>>> {text.replace(tokeyboi.mask_token, tokeyboi.decode([token]))}'")


'>>> A Gen Z person might say lol ur  say is so janky. '
'>>> A Gen Z person might say lol ur  that is so janky. '
'>>> A Gen Z person might say lol ur  coin is so janky. '
'>>> A Gen Z person might say lol ur  says is so janky. '
'>>> A Gen Z person might say lol ur  state is so janky. '
